# A full business solution

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

In [1]:
# Importing the Libraries

import requests
import json
import time
import re
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

In [2]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """
    def __init__(self, url):
        self.url = url
        response = requests.get(headers=headers, url=self.url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No Title Found"
        if soup.body:
            for irrelevent in soup.body(["script", "style", "img", "input"]):
                irrelevent.decompose()
            self.text = soup.body.get_text(separator='\n', strip=True)
        else:
            self.text = ""

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [3]:
test_url = Website("https://edwarddonner.com")
print(test_url.get_contents()[:100])
print(test_url.links)

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits 
['https://edwarddonner.com/', 'https://edwarddonner.com/connect-four/', 'https://edwarddonner.com/outsmart/', 'https://edwarddonner.com/about-me-and-about-nebula/', 'https://edwarddonner.com/posts/', 'https://edwarddonner.com/', 'https://news.ycombinator.com', 'https://nebula.io/?utm_source=ed&utm_medium=referral', 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html', 'https://patents.google.com/patent/US20210049536A1/', 'https://www.linkedin.com/in/eddonner/', 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/', 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/', 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/', 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/', 'https://edwarddonner.com/2025/04/21/th

# First step: Have Mistral figure out which links are relevant
Use a call to Mistral to read the links on a webpage, and respond in structured JSON.

It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".

We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [74]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages and don't give urls which is not starting with https.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [73]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
and don't give urls which is not starting with https


In [43]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [44]:
print(get_links_user_prompt(test_url))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [50]:
MODEL_NAME="mistral"  # Get this from you local ollama by using "ollama list"
OLLAMA_URL = "http://localhost:11434/api/chat"

In [51]:
def get_links(url):
    website = Website(url)
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        "stream": False  # Important: to get streaming responses
    }
    start_time = time.time()
    response = requests.post(OLLAMA_URL, json=payload, stream=False)
    print("Total Time Taken: ", time.time()-start_time)
    reply = response.json()["message"]["content"]
    reply = re.sub(r"<think>.*?</think>", "", reply, flags=re.DOTALL).strip()
    return reply

In [64]:
print(get_links("https://huggingface.co"))

Total Time Taken:  28.611915588378906
{
    "links": [
        {"type": "about page", "url": "https://huggingface.co"},
        {"type": "careers page", "url": "https://apply.workable.com/huggingface/"},
        {"type": "enterprise page", "url": "https://endpoints.huggingface.co"},
        {"type": "docs page", "url": "https://huggingface.co/docs"},
        {"type": "blog page", "url": "https://huggingface.co/blog"},
        {"type": "discussion forum", "url": "https://discuss.huggingface.co/"},
        {"type": "status page", "url": "https://status.huggingface.co/"},
        {"type": "github repository", "url": "https://github.com/huggingface"},
        {"type": "twitter profile", "url": "https://twitter.com/huggingface"},
        {"type": "linkedin company page", "url": "https://www.linkedin.com/company/huggingface/"}
    ]
}


## Second step: make the brochure!

In [65]:
def get_all_details(url):
    result = "Landing Page:\n"
    result += Website(url).get_contents()
    links = json.loads(get_links(url))  # converting the string output to JSON format
    print(f"Found Links: {links}")
    for link in links["links"]:
        result += f"\n\n{link["type"]}\n"
        result += Website(link["url"]).get_contents()
    return result

In [66]:
print(get_all_details("https://huggingface.co"))

Total Time Taken:  27.71848750114441
Found Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'docs', 'url': 'https://huggingface.co/docs'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co/'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'enterprise', 'url': 'https://endpoints.huggingface.co'}]}
Landing Page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Get started with Inference in seconds 🚀
Reachy Mini: The Open Robot for AI Builders
Welcome C

In [75]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [76]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20000]
    return user_prompt

In [77]:
print(get_brochure_user_prompt(company_name="Hugging Face", url="https://huggingface.co"))

Total Time Taken:  37.60173225402832
Found Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://endpoints.huggingface.co'}, {'type': 'docs', 'url': 'https://huggingface.co/docs'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community', 'url': 'https://discuss.huggingface.co/'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
You are looking at a company called: Hugging Face
Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.
Landing Page:
Webpage Title:
Hugging Face – The AI community building the futur

In [78]:
def create_brochure(company_name, url):
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        "stream": False  # Important: to get streaming responses
    }
    start_time = time.time()
    response = requests.post(OLLAMA_URL, json=payload, stream=False)
    print("Total Time Taken: ", time.time()-start_time)
    reply = response.json()["message"]["content"]
    reply = re.sub(r"<think>.*?</think>", "", reply, flags=re.DOTALL).strip()
    display(Markdown(reply))

In [79]:
create_brochure(company_name="Hugging Face", url="https://huggingface.co")

Total Time Taken:  32.013800859451294
Found Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://endpoints.huggingface.co'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co/'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'github repository', 'url': 'https://github.com/huggingface'}, {'type': 'twitter profile', 'url': 'https://twitter.com/huggingface'}, {'type': 'facebook page', 'url': 'https://www.facebook.com/HuggingFaceAI/'}]}
Total Time Taken:  85.63887929916382


Hugging Face is a technology company that specializes in artificial intelligence (AI) and machine learning. Some of their key products include the Transformers library for natural language processing, the Datasets platform for managing and sharing datasets, and the Model Hub for hosting and sharing pre-trained AI models. The company was founded in 2016 by Thierry François, Hugo Toulemonde, Audrey Touzeau, and Alexis Conneau, and is based in New York City.

In addition to their software offerings, Hugging Face also offers a range of AI-related services such as consulting, training, and research. They work with a variety of clients, including large corporations, startups, and academic institutions. The company has received funding from investors such as Accel, Y Combinator, and Andreessen Horowitz.

Hugging Face is known for its contributions to the development of large language models, such as the Turing-NLG model, which was one of the first high-performance open-source models for natural language generation. The company continues to develop and release new AI models, and their products are used by a wide range of organizations and individuals for tasks such as text generation, translation, and sentiment analysis.